7장에서 살펴본 내용을 떠올려보면 pandas의 `cut`,`qcut`이라는 메서드를 사용해서 선탣한 크기만큼 혹은 표본 변위치에 따라 데이터를 나눌수있었다.<br/>
이 함수를 `groupby`와 조합하면 데이터 묶음에 대해 변위치 분석이나 버킷 분석을 아주 쉽게 수행할 수 있다.<br/>
다음 임의의 데이터 묶음을 `cut`을 이용해서 등간격 구간으로 나누어보자.

In [1]:
frame = pd.DataFrame({'data1':np.random.randn(1000),
                     'data2':np.random.randn(1000)})

In [3]:
frame.tail()

,data1,data2
995,2.290473,0.032163
996,-0.350855,-0.205838
997,-0.710169,-0.316211
998,0.083800,-1.894766
999,0.525338,1.691662


In [4]:
factor = pd.cut(frame.data1, 4)

In [7]:
factor.value_counts()

(-0.126, 1.367]    469
(-1.62, -0.126]    397
(1.367, 2.861]      83
(-3.119, -1.62]     51
Name: data1, dtype: int64

In [9]:
factor[:10]

0    (-1.62, -0.126]
1    (-0.126, 1.367]
2    (-0.126, 1.367]
3    (-1.62, -0.126]
4    (-0.126, 1.367]
5    (-0.126, 1.367]
6    (-0.126, 1.367]
7    (-0.126, 1.367]
8    (-0.126, 1.367]
9    (-0.126, 1.367]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.119, -1.62] < (-1.62, -0.126] < (-0.126, 1.367] < (1.367, 2.861]]

`cut` 메서드는 Factor 객체를 반환하는데, 이 Factor 객체는 바로 groupby로 넘길 수 있다.<br/>
그래서 data2 컬럼에 대한 몇 가지 통계를 다음과 같이 계산할 수 있다.

In [15]:
frame.tail()

,data1,data2
995,2.290473,0.032163
996,-0.350855,-0.205838
997,-0.710169,-0.316211
998,0.083800,-1.894766
999,0.525338,1.691662


In [10]:
def get_stats(group):
    return {'min':group.min(),
           'max':group.max(),
           'count':group.count(),
           'mean':group.mean()}

In [16]:
grouped = frame['data2'].groupby(factor)

In [17]:
grouped

In [19]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.119, -1.62]",51.0,2.688303,0.063017,-2.499611
"(-1.62, -0.126]",397.0,2.805104,0.039959,-2.694253
"(-0.126, 1.367]",469.0,3.222946,-0.042999,-3.219478
"(1.367, 2.861]",83.0,2.188808,-0.081293,-2.013059


이는 등간격 버킷이고, 표본 변위치에 근거하여 크기가 같은 버킷을 계산하려면 qcut을 사용해야 한다.<br/>
`labels=False` 를 넘겨서 변위치 숫자를 구할 수도 있다.

In [41]:
grouping = pd.qcut(frame.data1, 10)

In [34]:
grouping.tail()

995      (1.252, 2.861]
996    (-0.495, -0.243]
997    (-0.858, -0.495]
998    (0.00588, 0.258]
999      (0.258, 0.552]
Name: data1, dtype: category
Categories (10, interval[float64]): [(-3.114, -1.266] < (-1.266, -0.858] < (-0.858, -0.495] < (-0.495, -0.243] ... (0.258, 0.552] < (0.552, 0.855] < (0.855, 1.252] < (1.252, 2.861]]

In [42]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [43]:
grouping.tail()

995    9
996    3
997    2
998    5
999    6
Name: data1, dtype: int64

In [37]:
grouping = frame.data2.groupby(grouping)

In [38]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.119, -1.62]",51.0,2.688303,0.063017,-2.499611
"(-1.62, -0.126]",397.0,2.805104,0.039959,-2.694253
"(-0.126, 1.367]",469.0,3.222946,-0.042999,-3.219478
"(1.367, 2.861]",83.0,2.188808,-0.081293,-2.013059
